In [ ]:
'''

# Author Copilot (c) 2023 by Matthew Favela
# Please read the README.md before running this notebook.

'''

### Load config and packages

In [ ]:
import os
import dotenv
import openai
import json
import pypandoc
import shutil

dotenv.load_dotenv()

# If you make changes to the `config.json` file, you will need to re-run this cell to load the new config.
with open('config.json') as f:
    config = json.load(f)
    

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

### Generate a title and table of contents based on the topic

In [ ]:
title_and_contents_prompt = f'''You are a professional on everything {str(config["topic"])}. You are here to write a table of contents for an eBook based on what i tell you to. You must follow my steps. You need to captivate the reader and keep their attention with a title that represents the book in the most quality and value maximizing way. Be insightful and talk as if you genuinely want to help them. The book will be {str(config["num_chapters"])} chapters long with sub chapters, sub sub chapters, and paragraphs. The table of contents must be in standard HTML format so that it can be exported in .epub format. use h1 for headers h2 for subheaders and so on and so forth. do not space out the tags, just have them line afer line but properly formatted. Do not put the tags for it to be a website (the following are unneccessary (DOCTYPE, html, header, body)), just format in html'''

title_and_contents = openai.ChatCompletion.create(
  model=config["model"],
  messages=[
        {
            "role": "system", 
            "content": title_and_contents_prompt}
    ],
    max_tokens=3500,
)

with open('book_draft.html', 'a') as f:
    f.write(title_and_contents['choices'][0]['message']['content'])
with open('title_and_contents.html', 'w') as f:
    f.write(title_and_contents['choices'][0]['message']['content'])
with open('log.txt', 'a') as f:
    f.write(title_and_contents['choices'][0]['message']['content'])

### Check the title_and_contents.html and make any necessary changes, then run the next cells

In [ ]:
title_and_contents = None
with open('title_and_contents.html', 'r') as f:
    title_and_contents = f.read()

In [ ]:
num_chapters = title_and_contents.count('<h2>')
toc_dict = {}
chapters = title_and_contents.split('<h2>')
chapter_names = []
chapters.pop(0)
for index, chapter in enumerate(chapters):
    chapter_name = chapter.split('</h2>')[0]
    chapter_names.append(chapter_name)

    sub_chapters = chapter.split('<h3>')
    sub_chapters.pop(0)
    sub_chapters_dict = {}
    for sub_chapter in sub_chapters:
        sub_chapter_name = sub_chapter.split('</h3>')[0]
        sub_sub_chapters = sub_chapter.split('<h4>')
        sub_sub_chapters.pop(0)
        sub_sub_chapters_dict = {}
        for sub_sub_chapter in sub_sub_chapters:
            sub_sub_chapter_name = sub_sub_chapter.split('</h4>')[0]
            paragraphs = sub_sub_chapter.split('<p>')
            paragraphs.pop(0)
            paragraphs_dict = {}
            for paragraph in paragraphs:
                paragraph_name = paragraph.split('</p>')[0]
                paragraphs_dict[paragraph_name] = ''
            sub_sub_chapters_dict[sub_sub_chapter_name] = paragraphs_dict
        sub_chapters_dict[sub_chapter_name] = sub_sub_chapters_dict
    toc_dict[f"Chapter {index}"] = sub_chapters_dict

In [ ]:
ai_sys_message = f'''You are a professional on everything {config["topic"]} and HTML. You are here to write a book based on what i tell you to. You must follow my steps. Be creative, witty, and passionate, but dont over do it. You need to captivate the reader and keep their attention. Be insightful and talk as if you genuinely want to help them. The book will be {str(config["num_chapters"])} so you can ramble and talk about subjects for a long time to soak up page space. Create a lot of value out of my book and even some call to actions. Each chapter will be 2500 words minimum.'''
def generate_chapter(chapter_expanded):
    chapter = f'''Write the chapter of an ebook based on the following information with a minimum of 2500 words for the entire chapters length and an absolute minimum of 5 sentences per paragraph (IMPORTANT: with <p> around each paragraph/body of text). Go into a lot of detail, be insightful, intelligent but talk in simple terms, and professional like you know what youre talking about and keep the reader interested and captivated to read more: table of contents {str(chapter_names)} chapter to write: {str(chapter_expanded)}. The chapter will be in standard HTML format using (<h1> tags for headers) (<h2> tags for sub headers) (<p> for paragraphs or bodies of text) and so on and so forth. every piece of text must have a HTML tag and formatted correctly for an eBook'''
    response = openai.ChatCompletion.create(
    model=config["model"],
    messages=[
        {"role": "system", "content": ai_sys_message},
        {"role": "user", "content": chapter}
        ],
    max_tokens=3000
    )
    with open('book_draft.html', 'a') as f:
        f.write("\n\n" + response['choices'][0]['message']['content'])
    with open('log.txt', 'a') as f:
        f.write("\n\n\n\n\n\n\n-----------------------------------\n")
        f.write(response['choices'][0]['message']['content'])
        f.write("\n-----------------------------------")
    return response['choices'][0]['message']['content']

### Generate Introduction Chapter

In [ ]:
print(f"-- Generating introduction chapter --")
intro_chapter = generate_chapter("Introduction chapter")
print(f"-- Successfully generated introduction chapter --")

### Generate chapters

In [ ]:
for chapter_number in range(config["num_chapters"]):
    if(chapter_number == 0):
        continue
    print(f"-- Generating chapter {str(chapter_number)} --")
    chapter = generate_chapter(toc_dict[f"Chapter {str(chapter_number)}"])
    print(f"-- Successfully generated chapter {str(chapter_number)} --")

### Generate Bonus Chapter

In [ ]:
print(f"-- Generating bonus chapter --")
chapter_bonus = generate_chapter("BONUS Chapter")
print(f"-- Successfully generated bonus chapter --")

### Generate Conclusion Chapter

In [ ]:
print(f"-- Generating conclusion chapter --")
chapter_conclusion = generate_chapter("Conclusion chapter")
print(f"-- Successfully generated conclusion chapter --")

### Generate the eBook

In [ ]:
pypandoc.convert_file("book_draft.html", 'epub', outputfile=f"{config['topic']}-{config['author']}.epub")
#move into eBook-Saves folder
shutil.move(f"{config['topic']}-{config['author']}.epub", f"eBook-Saves/{config['topic']}-{config['author']}.epub")
